In [56]:
from datapreparation import *
from simsiam import *
from utils import *
from evaluation import *
import torch
from collections import OrderedDict
from torch.utils.data import DataLoader
from torch.utils.data import Dataset, TensorDataset, ConcatDataset
from torchvision import datasets, transforms
import torch.optim as optim
import copy

https://github.com/vaseline555/Federated-Averaging-PyTorch/tree/1afb2be2c1972d8527efca357832f71c815b30b4/src

In [17]:
class TwoCropsTransform:
    """Take two random crops of one image as the query and key."""

    def __init__(self, base_transform):
        self.base_transform = base_transform

    def __call__(self, x):
        q = self.base_transform(x)
        k = self.base_transform(x)
        return [q, k]
    

def create_datasets(num_clients, iid):
    """Split the whole dataset in IID or non-IID manner for distributing to clients."""
    # get train and test dataset from cifar-10
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

    # MoCo v2's aug: similar to SimCLR https://arxiv.org/abs/2002.05709
    augmentation = [
        transforms.RandomResizedCrop(224, scale=(0.2, 1.)),
        transforms.RandomApply([
            transforms.ColorJitter(0.4, 0.4, 0.4, 0.1)  # not strengthened
        ], p=0.8),
        transforms.RandomGrayscale(p=0.2),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize
    ]

    trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=TwoCropsTransform(transforms.Compose(augmentation)))
    testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                        download=True, transform=transforms.Compose([transforms.ToTensor(), normalize]))

    if iid:
        shuffled_indices = torch.randperm(len(trainset))

        training_inputs = trainset.data[shuffled_indices]
        training_labels = torch.Tensor(trainset.targets)[shuffled_indices]
        split_size = len(trainset) // num_clients
        split_datasets = list(
                    zip(
                        torch.split(torch.Tensor(training_inputs), split_size),
                        torch.split(torch.Tensor(training_labels), split_size)
                    )
                )
        local_trainloaders = [
                    torch.utils.data.DataLoader(local_dataset, batch_size=4, 
                                                    shuffle=True, num_workers=2, pin_memory=True)
                    for local_dataset in split_datasets
    ]
    else: 
        pass

    testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                            shuffle=False, num_workers=2, pin_memory=True)
    return local_trainloaders, testloader

In [55]:
class Client:
    def __init__(self, client_id, model, dataloader, batchsize, local_epochs, device):
        self.client_id = client_id
        self.dataloader = dataloader
        self.model = model
        self.optimizer = optim.SGD(self.model.parameters(), lr=0.03, momentum=0.9, weight_decay=0.0005)
        self.local_epochs = local_epochs
        self.device = device


    def client_update(self):
        self.model.train()
        self.model.to(self.device)
        optimizer = self.optimizer

        for epoch in range(self.local_epochs):  # loop over the dataset multiple times
            epoch_loss = 0.0
            running_loss = 0.0
            for i, data in enumerate(self.dataloader):            
                # get the inputs; data is a list of [inputs, labels]
                # inputs, labels = data
                images, _ = data[0], data[1].to(self.device)
                # zero the parameter gradients
                optimizer.zero_grad()

                # get the two views (with random augmentations):
                x1 = images[0].to(self.device)
                x2 = images[1].to(self.device)
                
                # forward + backward + optimize
                z1, p1 = self.model(x1)
                z2, p2 = self.model(x2)
                #loss = criterion(outputs, labels)
                loss = D(p1, z2)/2 + D(p2, z1)/2
                loss.backward()
                optimizer.step()

                # print statistics
                running_loss += loss.item()
                epoch_loss += loss.item()
                if i % 100 == 99:    # print every 2000 mini-batches
                    print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 100:.3f}')
                    running_loss = 0.0
            print("epoch loss = ", epoch_loss/len(self.dataloader))
        print('Finished Training')

    def client_evaluate(self):
        """evaluates model on local dataset TODO: Should this be done in self-supervised learning and if so, how?"""
        # insert evaluate() method of SimSiam
        pass

In [57]:
class Server:
    def __init__(self, num_clients, iid, num_rounds):
        self.num_clients = num_clients
        self.iid = iid
        self.num_rounds = num_rounds

    def setup(self):
        local_trainloaders, test_loader = create_datasets(self.num_clients, self.iid)
        self.device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
        self.clients = self.create_clients(local_trainloaders)
        self.testloader = test_loader
        self.send_model()
        
    def create_clients(self, local_trainloaders):
        clients = []
        for i, dataloader in enumerate(local_trainloaders):
            client = Client(client_id=i, model=SimSiam().to(self.device), dataloader=dataloader, batchsize=4, local_epochs=5, device=self.device)
            clients.append(client)
        return clients

    def send_model(self):
        """Send the updated global model to selected/all clients."""
        for client in self.clients:
            client.model = copy.deepcopy(self.model)

    def average_model(self):
        """Average the updated and transmitted parameters from each selected client."""
        averaged_weights = OrderedDict()

        for i, client in enumerate(self.clients):
            local_weights = client.model.state_dict()

            for key in self.model.state_dict().keys():
                if i == 0:
                    averaged_weights[key] = coefficients[it] * local_weights[key]
                else:
                    averaged_weights[key] += coefficients[it] * local_weights[key]
        self.model.load_state_dict(averaged_weights)


    def train_federated_model(self):
        self.send_model()
        self.average_model()
    
    def evaluate_global_model(self):
        # insert evaluation function here
        pass

    def main(self):
        for i in range(self.num_rounds):
            self.train_federated_model()
            # test_loss, test_accuracy = self.evaluate_global_model() # TODO
        self.send_model()

In [52]:
server = Server(num_clients=5, iid=True)
server.setup()

Files already downloaded and verified
Files already downloaded and verified


/home/jonas/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [8]:
# PATH = "models/simsiam.pth"

# # load trained model
# model = SimSiam()
# device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# model.load_state_dict(torch.load(PATH, map_location=device))